<a href="https://colab.research.google.com/github/PhoenixCC0722/Journey_to_become_DataScientist/blob/main/10_advanced_transformations__sql_to_pd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# From SQL to pandas challenge 10

In [1]:
# import libraries
import pandas as pd
import numpy as np

# load data
# This code is made to load our data stored on Google Drive
def gd_path(file_id):
    """Generate a shareable link from Google Drive file id."""
    return f"https://drive.google.com/uc?export=download&id={file_id}"

# Google Drive file ids
files_id = {
    "titles": "1PLdn50N9GRa53ZbuVWo0l47F_IXdvlEm",
    "sales": "1fzFc9rwYmVIPaGOFmhLVxCi3kg19vNU2",
}

# Read data from Google Drive
sales = pd.read_csv(gd_path(files_id["sales"]), sep=";")
titles = pd.read_csv(gd_path(files_id["titles"]), sep=";")

## 1. Select everything from the sales table and create a new column called "sales_category" to categorise qty:
   
		qty >= 50 high sales
		20 <= qty < 50 medium sales
		qty < 20 low sales

In [2]:
df = pd.DataFrame(sales)
df.head()

,stor_id,ord_num,ord_date,qty,payterms,title_id
0,6380,6871,1994-09-14 00:00:00,5,Net 60,BU1032
1,6380,722a,1994-09-13 00:00:00,3,Net 60,PS2091
2,7066,A2976,1993-05-24 00:00:00,50,Net 30,PC8888
3,7066,QA7442.3,1994-09-13 00:00:00,75,ON invoice,PS2091
4,7067,D4482,1994-09-14 00:00:00,10,Net 60,PS2091


In [10]:
# define conditions for sales category
conditions = [
    (df['qty'] >= 50),
    (df['qty'] >= 20) & (df['qty'] < 50),
    (df['qty'] < 20)
]
# define the labels for different category
labels = ['high sales','medium sales','low sales']
# create a new column
df['sales_category'] = np.select(conditions, labels, default='')
print(df.head(10))


   stor_id   ord_num             ord_date  qty    payterms title_id  \
0     6380      6871  1994-09-14 00:00:00    5      Net 60   BU1032   
1     6380      722a  1994-09-13 00:00:00    3      Net 60   PS2091   
2     7066     A2976  1993-05-24 00:00:00   50      Net 30   PC8888   
3     7066  QA7442.3  1994-09-13 00:00:00   75  ON invoice   PS2091   
4     7067     D4482  1994-09-14 00:00:00   10      Net 60   PS2091   
5     7067     P2121  1992-06-15 00:00:00   40      Net 30   TC3218   
6     7067     P2121  1992-06-15 00:00:00   20      Net 30   TC4203   
7     7067     P2121  1992-06-15 00:00:00   20      Net 30   TC7777   
8     7131   N914008  1994-09-14 00:00:00   20      Net 30   PS2091   
9     7131   N914014  1994-09-14 00:00:00   25      Net 30   MC3021   

  sales_category  
0      low sales  
1      low sales  
2     high sales  
3     high sales  
4      low sales  
5   medium sales  
6   medium sales  
7   medium sales  
8   medium sales  
9   medium sales  


### Hint:

In SQL the syntax is:

```sql
SELECT *,
CASE
    WHEN qty >= 50 THEN "high sales"
    WHEN qty >= 20 THEN "medium sales"
    ELSE "low sales"
END AS sales_category
FROM sales;
```

## 2. Adding to your answer from the previous question. Find out the total amount of books sold (qty) in each sales category
    i.e. How many books had high sales, how many had medium sales, and how many had low sales

In [11]:
# define conditions for sales category
conditions = [
    (df['qty'] >= 50),
    (df['qty'] >= 20) & (df['qty'] < 50),
    (df['qty'] < 20)
]
# define the labels for different category
labels = ['high sales','medium sales','low sales']
# create a new column
df['sales_category'] = np.select(conditions, labels, default='')
filt_b = df.groupby('sales_category')['qty'].sum()
print(filt_b)

sales_category
high sales      125
low sales        83
medium sales    285
Name: qty, dtype: int64


### Hint:

In SQL the syntax is:

```sql
SELECT sum(qty),
CASE
	WHEN qty>=50 THEN 'high sales'
    WHEN (qty>=20 AND qty<50) THEN 'medium sales'
    ELSE 'low sales'
END AS sales_category
FROM sales
GROUP BY sales_category;
```

## 3. Adding to your answer from the previous questions: output only those sales categories that have a SUM(qty) greater than 100, and order them in descending order

In [13]:
# define conditions for sales category
conditions = [
    (df['qty'] >= 50),
    (df['qty'] >= 20) & (df['qty'] < 50),
    (df['qty'] < 20)
]
# define the labels for different category
labels = ['high sales','medium sales','low sales']
# create a new column
df['sales_category'] = np.select(conditions, labels, default='')
filt_b = df.groupby('sales_category').agg(sum_qty=('qty','sum')).reset_index()
filt_b = filt_b[filt_b['sum_qty'] > 100].sort_values(by='sum_qty',ascending=False)
print(filt_b)

  sales_category  sum_qty
2   medium sales      285
0     high sales      125


### Hint:

In SQL the syntax is:

```sql
SELECT sum(qty),
CASE
    WHEN qty>=50 THEN 'high sales'
    WHEN (qty>=20 AND qty<50) THEN 'medium sales'
    ELSE 'low sales'
END AS sales_category
FROM sales
GROUP BY sales_category
HAVING sum(qty)>100
ORDER BY sum(qty) DESC;
```

## 4. Find out the average book price, per publisher, for the following book types and price categories:
		book types: business, traditional cook and psychology
		price categories: <= 5 super low, <= 10 low, <= 15 medium, > 15 high
        
        - When displaying the average prices, use ROUND() to hide decimals.

In [27]:
df = pd.DataFrame(titles)
df.head()
df.dtypes
# define conditions for the price category
conditions = [
    (df['price'] <= 5),
    (df['price'] <= 10) & (df['price'] > 5),
    (df['price'] <= 15) & (df['price'] > 10),
    (df['price'] > 15)
]
labels = ['super low','low','medium','high']
df['price_category'] = np.select(conditions, labels, default='')
filt_df = df[df['type'].isin(['business','traditional cook','psychology'])]
avg_price = filt_df.groupby(['pub_id','price_category'])['price'].mean().round(0).reset_index()
print(avg_price)

   pub_id price_category  price
0     736           high   20.0
1     736            low    7.0
2     736         medium   11.0
3     736      super low    3.0
4     877           high   22.0
5    1389           high   20.0
6    1389         medium   12.0


### Hint:

In SQL the syntax is:

```sql
SELECT
    ROUND(AVG(price)),
    type,
    pub_id,
CASE
    WHEN price <= 5 THEN 'super low'
    WHEN (price > 5 AND price <= 10) THEN 'low'
    WHEN (price > 10 AND price <= 15) THEN 'medium'
    ELSE 'high'
END AS price_category
FROM titles
GROUP BY
    pub_id,
    type,
    price_category
HAVING
    type IN ('business', 'trad_cook', 'psychology');
```